In [ ]:
from galfind 

In [ ]:
import matplotlib.patheffects as PathEffects
import numpy as np
from astropy.table import Table
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmasher 
import os
import glob
plt.rcParams["figure.dpi"] = 300

from EXPANSE import ResolvedGalaxy, MultipleResolvedGalaxy
# import LogNorm

from matplotlib.colors import LogNorm

In [ ]:
galaxies = ResolvedGalaxy.init_all_field_from_h5('JOF_psfmatched', '/nvme/scratch/work/tharvey/EXPANSE/galaxies/')

In [ ]:
for galaxy in galaxies:
    for run_name in ['photoz_delayed', 'photoz_lognorm', 'photoz_dpl', 'photoz_continuity']:
        galaxy.load_bagpipes_results(run_name, run_dir = '/nvme/scratch/work/tharvey/EXPANSE/pipes/')

In [ ]:
mult_galaxy = MultipleResolvedGalaxy(galaxies)

print([galaxy.get_number_of_bins() for galaxy in mult_galaxy])

number_of_bins = np.array([galaxy.get_number_of_bins() for galaxy in mult_galaxy])
mask = np.array(number_of_bins) > 1
mask_2bins = np.array(number_of_bins) == 2
print(f'{np.sum(number_of_bins[mask])} bins in total')
print(f'{np.sum(mask)} galaxies with more than 1 bin')
print(f'{np.sum(mask_2bins)} galaxies with 2 bins')
number_of_bins_dict = {gal.galaxy_id: gal.get_number_of_bins() for gal in mult_galaxy}
print(number_of_bins_dict)

In [ ]:
#files = [galaxy.h5_path for galaxy in mult_galaxy if galaxy.get_number_of_bins() == 1]
#for file in files:
#    os.rename(file, file.replace('JOF_psfmatched', 'singlebin/JOF_psfmatched'))


plot_paths = [f'/nvme/scratch/work/tharvey/EXPANSE/galaxies/diagnostic_plots/JOF_psfmatched_{galaxy.galaxy_id}_overview.png' for galaxy in mult_galaxy]

for galaxy, plot_path in zip(mult_galaxy, plot_paths):
    if galaxy.get_number_of_bins() == 1:
        os.rename(plot_path, plot_path.replace('JOF_psfmatched', 'singlebin/JOF_psfmatched'))

In [ ]:

files = glob.glob('/nvme/scratch/work/tharvey/EXPANSE/galaxies/diagnostic_plots/JOF_psfmatched_*_overview.png')

for file in files:
    galaxy_id = file.split('_')[3]
    if galaxy_id not in number_of_bins_dict.keys():
        print(galaxy_id)
        os.rename(file, file.replace('JOF_psfmatched', 'lowz/JOF_psfmatched'))

In [ ]:
path_cat = "/raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/ACS_WFC+NIRCam/JOF_psfmatched/JOF_psfmatched_MASTER_Sel-F277W+F356W+F444W_v11_total.fits"

cat = Table.read(path_cat)
print(len(cat))
masks = [col for col in cat.colnames if "unmasked_" in col]
for mask in masks:
    cat = cat[cat[mask]]
print(len(cat))

# Define the selection criteria
cat = cat[cat["EPOCHS"]]
def selection(c):
    return (c["EPOCHS"]) & (c["zbest_fsps_larson_zfree"] > 4.5) #(c["MAG_APER_F277W_TOTAL"] < 29) & (c["kron_radius_F277W+F356W+F444W"] > 10) &


# Apply the selection



mask = selection(cat)
cat_selected = cat[mask]
print(f'Number of galaxies selected {len(cat_selected)}')
color_by = "zbest_fsps_larson_zfree"

# get number of bins for selected galaxies only

number_of_bins = [number_of_bins_dict[str(x)] if str(x) in number_of_bins_dict.keys() else 0 for x in cat_selected["NUMBER"] ]

fig, ax = plt.subplots(1, 1, figsize=(4, 4), facecolor="white", dpi = 300)
cax = make_axes_locatable(ax).append_axes("right", size="5%", pad=0.05)
# Add top axis
ax.set_xscale("log")

ax_top = ax.secondary_xaxis("top", functions=(lambda x: x * 0.03, lambda x: x / 0.03))
ax_top.set_xlabel("Kron Radius [arcsec]")
ax.set_xlabel("Kron Radius [pixels]")
ax.set_ylabel(r"F277W Total Magnitude [AB]")

#assert len(cat_selected) == len(number_of_bins_ordered)

points = ax.scatter(
    cat_selected["kron_radius_F277W+F356W+F444W"],
    cat_selected["MAG_APER_F277W_TOTAL"],
    s=10,
    c=number_of_bins,
    cmap="cmr.guppy",
    alpha=0.9,
    edgecolor="black",
    linewidth=0.9,
    norm = LogNorm()
)

'''points = ax.scatter(
    cat["kron_radius_F277W+F356W+F444W"],
    cat["MAG_APER_F277W_TOTAL"],
    s=10,
    c=cat[color_by],
    cmap="cmr.guppy",
    alpha=0.9,
    edgecolor="black",
    linewidth=0.9,
)
'''
ax.set_ylim(30.5, 19)
ax_top.set_xscale("log")
def y(x):
    return -8.03979 * np.log10(x) + 33.88689  # LOBF for stars
x = np.linspace(10, 150, 100)

ax.plot(
    x,
    y(x),
    c="yellow",
    zorder=1,
    path_effects=[PathEffects.withStroke(linewidth=3, foreground="black")],
    linestyle="dashed",
)
fig.colorbar(points, cax=cax, label="Number of bins")
# make colorbar log scale
cax.yaxis.set_major_formatter(ScalarFormatter())
cax.set_yticks([1, 10, 80])


ax.xaxis.set_major_formatter(ScalarFormatter())
ax.set_xticks([5, 10, 20, 50, 100])
ax_top.xaxis.set_major_formatter(ScalarFormatter())
ax_top.set_xticks([0.15, 0.3, 0.6, 1.5, 4.5])
ax.axvline(10, color="black", linestyle="--")
ax.axhline(29, color="black", linestyle="--")


stars = Table.read("/nvme/scratch/work/tharvey/catalogs/JOF_psfmatched_PSF_STARS.fits")
ax.scatter(
    stars["kron_radius_F277W+F356W+F444W_1a"],
    stars["MAG_APER_F277W_TOTAL"],
    s=60,
    color="yellow",
    alpha=0.9,
    marker="*",
    edgecolor="black",
    linewidth=1,
)


'''
plt.savefig(
    "/nvme/scratch/work/tharvey/resolved_sedfitting/plots/kron_radius_vs_mag.png",
    dpi=300,
    bbox_inches="tight",
)
'''
plt.show()

ids_selected = list(cat_selected["NUMBER"].data)
print(ids_selected)
min_cutout_size = 64
cat_selected["CUTOUT_SIZE"] = 2.5 * cat_selected["kron_radius_F277W+F356W+F444W"]
cat_selected["CUTOUT_SIZE"] = np.maximum(cat_selected["CUTOUT_SIZE"], min_cutout_size)
# Round them all to be odd

cat_selected["CUTOUT_SIZE"] = np.ceil(cat_selected["CUTOUT_SIZE"]) // 2 * 2 + 1
'''
cat_selected.write(
    "/nvme/scratch/work/tharvey/EXPANSE/catalogs/JOF_psfmatched_MASTER_Sel-F277W+F356W+F444W_v11_total_selected_v2.fits",
    overwrite=True,
)
'''



In [ ]:
plt.hist(cat_selected["zbest_fsps_larson_zfree"], bins=20)

print(np.min(cat_selected["zbest_fsps_larson_zfree"]))

In [ ]:
colors = ['red', 'blue', 'green', 'purple']
fig, ax = plt.subplots(1, 1, figsize=(4, 4), facecolor="white", dpi = 300)
labelled = False
for galaxy in galaxies:
    base_redshift = galaxy.redshift
    for pos, run_name in enumerate(['photoz_delayed', 'photoz_lognorm', 'photoz_dpl', 'photoz_continuity']):
        pipes_redshift = galaxy.sed_fitting_table['bagpipes'][run_name]['redshift_50'][0]
        ax.scatter(base_redshift, pipes_redshift, color = colors[pos], label = run_name if not labelled else '', s = 10)
    labelled = True
        
# Plot a 1:1 line between 4.5 and 10.5

ax.plot([4.5, 10.5], [4.5, 10.5], color = 'black', linestyle = '--')
ax.legend()
ax.set_xlabel('EAZY redshift')
ax.set_ylabel('BAGPIPES redshift')
